## Predicting Probability of Loan Delinquency

#### In this assignment, we will explore the historical loan data from [lending club](https://www.kaggle.com/wendykan/lending-club-loan-data/data), and build a very short machine learning pipeline to predict on likelihood of loan delinquency. 

#### We will start out with inspecting and describing the data, then prepare the dataset for a K-Nearest Neighbor algorithm, and evaluate the performance of the algorithm. This exercise walks you through _most_ steps of a machine learning project with much simplicity and focus on data manipulation rather than algorithm training. You are encouraged to check out the [sklearn library](http://scikit-learn.org/stable/index.html) and the recommended reading materials this week to learn more about ML!

#### There are 17 questions in total. Each question worths 5 points. Proper documentation and readability of your code will be 15 points.

In [32]:
import pandas as pd
import numpy as np
pd.options.display.max_columns = 999
pd.set_option('display.float_format', lambda x: '%.3f' % x)

### Inspect the data 

In [33]:
loan = pd.read_csv('loan.csv')



/usr/local/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2728: DtypeWarning: Columns (19,55) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [34]:
print(loan.shape)

(887379, 74)


In [35]:
print(list(loan))

['id', 'member_id', 'loan_amnt', 'funded_amnt', 'funded_amnt_inv', 'term', 'int_rate', 'installment', 'grade', 'sub_grade', 'emp_title', 'emp_length', 'home_ownership', 'annual_inc', 'verification_status', 'issue_d', 'loan_status', 'pymnt_plan', 'url', 'desc', 'purpose', 'title', 'zip_code', 'addr_state', 'dti', 'delinq_2yrs', 'earliest_cr_line', 'inq_last_6mths', 'mths_since_last_delinq', 'mths_since_last_record', 'open_acc', 'pub_rec', 'revol_bal', 'revol_util', 'total_acc', 'initial_list_status', 'out_prncp', 'out_prncp_inv', 'total_pymnt', 'total_pymnt_inv', 'total_rec_prncp', 'total_rec_int', 'total_rec_late_fee', 'recoveries', 'collection_recovery_fee', 'last_pymnt_d', 'last_pymnt_amnt', 'next_pymnt_d', 'last_credit_pull_d', 'collections_12_mths_ex_med', 'mths_since_last_major_derog', 'policy_code', 'application_type', 'annual_inc_joint', 'dti_joint', 'verification_status_joint', 'acc_now_delinq', 'tot_coll_amt', 'tot_cur_bal', 'open_acc_6m', 'open_il_6m', 'open_il_12m', 'open_il

In [36]:
print(loan['member_id'][0:5])
print(loan['id'][0:5])

s = set()
for i in loan['member_id']:
    s.add(i)

print(len(s))

0    1296599
1    1314167
2    1313524
3    1277178
4    1311748
Name: member_id, dtype: int64
0    1077501
1    1077430
2    1077175
3    1076863
4    1075358
Name: id, dtype: int64
887379


In [37]:
u_mid = loan.member_id.unique()
print(len(u_mid))
print(max(u_mid), min(u_mid))

887379
73544841 70473


In [38]:
uid = loan.id.unique()
print(len(uid))
print(max(uid), min(uid))

887379
68617057 54734


In [39]:
issue_dates = loan['issue_d']
print(max(issue_dates), min(issue_dates))

Sep-2015 Apr-2008


#### Question 1: Browse the data dictionary and use pandas functions to browse the data and answer the following questions with a paragraph. You don't need to show your work :
    - How many columns are there? What general categories of information do they provide? 
    - What’s the time frame of the dataset based on the issue date of the loans? (You will have to parse the date) 
    - What’s the difference between “Id” and “Member Id”? 
    - How many unique borrowers and loans are in the dataset?

#### write a summary paragraph here: 

The "Loan.csv" file has 74 columns. The general categories are loan amount, location of person, past loan performance, where they got the loan at.  
The timeframe of the data set is between April 2008 to September 2015.
The number of unique Id and Member Id are the same. The maximum integer value of member id is higher than id and the minimun value is lower than id. 
There are 887379 borrowers and 887379 loans. 

### Descriptive statistics - About the clients

In [40]:
print(loan['annual_inc'].mean())

75027.5877608


####  What's the average annual individual income of the borrowers?

In [41]:
75027.5877608

75027.5877608

#### What are the top ten purposes people borrow money for? 

In [42]:
mapping = {}
for purpose in loan['purpose']:
    if purpose in mapping:
        mapping[purpose] += 1
    else:
        mapping[purpose] = 1

import operator
sorted_x = sorted(mapping.items(), key=operator.itemgetter(1))
for idx, e in enumerate(sorted_x):
    if idx == 10:
        break
    print(idx+1, e)

1 ('educational', 423)
2 ('renewable_energy', 575)
3 ('wedding', 2347)
4 ('house', 3707)
5 ('vacation', 4736)
6 ('moving', 5414)
7 ('medical', 8540)
8 ('car', 8863)
9 ('small_business', 10377)
10 ('major_purchase', 17277)


#### Which 10 states has the most number of loans? Sort by descending order

In [43]:
mapping = {}
for s in loan['addr_state']:
    if s in mapping:
        mapping[s] += 1
    else:
        mapping[s] = 1

sorted_x = sorted(mapping.items(), key=operator.itemgetter(1), reverse=True)
for idx, e in enumerate(sorted_x):
    if idx == 10:
        break
    print(idx+1, e) 

1 ('CA', 129517)
2 ('NY', 74086)
3 ('TX', 71138)
4 ('FL', 60935)
5 ('IL', 35476)
6 ('NJ', 33256)
7 ('PA', 31393)
8 ('OH', 29631)
9 ('GA', 29085)
10 ('VA', 26255)


### Descriptive statistics - About the loans

#### How many loans were issued per year?

In [44]:
mapping = {}

issue_d = loan['issue_d']
for d in issue_d:
    year = d.split('-')[1]
    if year in mapping:
        mapping[year] += 1
    else:
        mapping[year] = 1

for k in mapping:
    print(k, mapping[k])

2011 21721
2010 12537
2009 5281
2008 2393
2007 603
2013 134755
2012 53367
2014 235628
2015 421094


#### What’s the average interest rate for loans in each grade and within that for each sub_grade? 

In [51]:
grades = loan.grade.unique()
print(grades)
subgrades = loan.sub_grade.unique()
print(subgrades)

mapping = {}
for grade in grades:
    mapping[grade] = {}
    
for subgrade in subgrades:
    grade = subgrade[0]
    mapping[grade][subgrade] = []

print(mapping)

['B' 'C' 'A' 'E' 'F' 'D' 'G']
['B2' 'C4' 'C5' 'C1' 'B5' 'A4' 'E1' 'F2' 'C3' 'B1' 'D1' 'A1' 'B3' 'B4' 'C2'
 'D2' 'A3' 'A5' 'D5' 'A2' 'E4' 'D3' 'D4' 'F3' 'E3' 'F4' 'F1' 'E5' 'G4' 'E2'
 'G3' 'G2' 'G1' 'F5' 'G5']
{'B': {'B2': [], 'B5': [], 'B1': [], 'B3': [], 'B4': []}, 'C': {'C4': [], 'C5': [], 'C1': [], 'C3': [], 'C2': []}, 'A': {'A4': [], 'A1': [], 'A3': [], 'A5': [], 'A2': []}, 'E': {'E1': [], 'E4': [], 'E3': [], 'E5': [], 'E2': []}, 'F': {'F2': [], 'F3': [], 'F4': [], 'F1': [], 'F5': []}, 'D': {'D1': [], 'D2': [], 'D5': [], 'D3': [], 'D4': []}, 'G': {'G4': [], 'G3': [], 'G2': [], 'G1': [], 'G5': []}}


In [55]:
int_rates = loan['int_rate']

for idx, subgrade in enumerate(loan['sub_grade']):
    int_rate = loan['int_rate'][idx]
    grade = subgrade[0]
    mapping[grade][subgrade].append(int_rate)

for grade in mapping:
    grade_avg_int_list = []
    for sg in mapping[grade]:
        grade_avg_int_list += mapping[grade][sg]
        subgrade_avg_int = sum(mapping[grade][sg]) / float(len(mapping[grade][sg]))
        print(grade, sg, subgrade_avg_int)
        
    grade_avg_int = sum(grade_avg_int_list) / float(len(grade_avg_int_list))
    print(grade, grade_avg_int)
    print("\n")
    print("\n")

B B2 9.99664336522
B B5 12.2691690046
B B1 8.94879124787
B B3 10.9007918613
B B4 11.7448662496
B 10.8296181665




C C4 14.598234644
C C5 15.3181527936
C C1 12.8921316051
C C3 14.0159937003
C C2 13.4235697603
C 13.980098308




A A4 7.52918131534
A A1 5.71201850479
A A3 7.14102016456
A A5 8.2696195555
A A2 6.42585901712
A 7.2433115613




E E1 18.8276696956
E E4 20.780979188
E E3 19.9682418282
E E5 21.6807968668
E E2 19.3739884733
E 19.8973216887




F F2 23.3167433234
F F3 23.9705594406
F F4 24.4894925198
F F1 22.6062177889
F F5 24.9989090208
F 23.5827866007




D D1 16.079182626
D D2 16.8473925444
D D5 18.4172051054
D D3 17.3409072079
D D4 17.9032471242
D 17.1758144502




G G4 25.622428356
G G3 25.8429765545
G G2 25.6756723891
G G1 25.4576910743
G G5 25.6934548611
G 25.6267061396






#### What's the percentage breakdown of loan status? In particular we want to see the percentage of loans that were defaulted and fully paid.

In [63]:
mapping = {}
status = loan['loan_status']
total = float(len(status))
for s in status:
    if s in mapping:
        mapping[s] += 1
    else:
        mapping[s] = 1

for s in mapping:
    print(s, str(round((mapping[s]/total * 100),2)) + "%")

Fully Paid 23.41%
Charged Off 5.1%
Current 67.82%
Default 0.14%
Late (31-120 days) 1.31%
In Grace Period 0.7%
Late (16-30 days) 0.27%
Does not meet the credit policy. Status:Fully Paid 0.22%
Does not meet the credit policy. Status:Charged Off 0.09%
Issued 0.95%


Fully Paid: 23.41%
Defaulted: 0.14%

#### How does default rate varies from year to year? Create one dataframe showing year vs default rate

### Case Study

#### You are considering approval of a loan of 100,000 dollars to a couple whose joint income is 200,000 dollars. You've verfied their information and now you are considering whether you want to approve this loan offer and what interest/term you would offer based on historical data. Write a short paragraph about how you would decide if you want to approve the loan or not based on available data on borrowers such as income, employment histories, and so on. You are encouraged to but not required to include numbers you obtained from analyzing historical data. You may also discuss other uncertainties you would like to consider to justify your decision. 

### Machine Learning

#### Machine learning algorithms allow us to take in large quantity of information, identify patterns, and make data-driven decisions. Now we want to use it to aid our decision-making by predicting a client's likelihood of deliquency. 

#### Some lingos before we start:
__features__: also your regressor, left-hand variables, variables that you use to predict your outcome 

__target variable__: also outcome variable, variable whose value you are trying to predict. 



### A machine learning pipeline generally consists of :

__Data preprocessing__: clean the data by getting rid of outliers and idiosyncracies, standarizeding the data on the same scale, and creating target variable if it's not clearly defined by the problem.
    

#### Then an re-iterative process of:

- __Feature generation and selection__: create predictors, ie features, to provide "hints" to the model and select the most informative features.


- __Cross-validation__: divide the dataset into training and testing to obtain various performance metrics for the algorithms. Common partition scheme for this are 10-/5-fold cross validation. Read more [here](https://genome.tugraz.at/proclassify/help/pages/XV.html)


- __Algorithm selection/Grid search__: find the best performing algorithm and the best hyper-parameters for the algorithm. this process can be time consuming, which is the so-called "no free lunch thereom", but generally the process is quite automated.



- __Evaluation__: choose your loss function and metrics you want to maximize based on your real concerns and tradeoffs between false positive/false negative rate, training time/latency, and so on.


- __Implementation__: In a real project, what algorithm you choose is also informed by accessiblity to the data in real time, the required data literacy from the users/complexity of your algorithm, transparency/ethics, costs of maintaince, and so on. You will also build in check-ins for abnormal prediction results.

#### In the following section you will get the data ready to feed it through a machine learning pipeline.

In [ ]:
# read in your data frame here
# warning: this will update the df global variable 
# and erase the columns you created previously for descriptive analysis! we want to start fresh here.

#### First of all, we will sub-set our dataframe. To keep the training time short we will only use data from 2015. Then let's divide it into two parts: historical records with known payment outcomes vs loans that are currently active or just issued. We will use the historical records (with known outcomes) to train, test, and evaluate our model. In reality, we will apply the trained model to predict on loans that are still active (outcomes unkown) . 

__Task__: Create one df that contains all loans in 2015 then split it into two dataframes: one with status that are either 'Current' or 'Issued' - call it _application_, and one that contains the rest -- call it _development_

#### Now we'll work with our _development set_. 

#### We will adjust our outcome variable slightly -- instead of predicting on default rate which is quite low and can make accurate prediction more challenging, we will predict on "delinquency" which include loans that are charged off, late, and defaulted. In this way, we increase the number of outcomes and also simplifies the problem into a binary classification -- meaning we have two outcomes essentially: delinquent (1) / not delinquent (0)

__Task:__ Create a new column called "delinquency" that consolidate the values from "loan_status" and is a binary variable that is 1 for delinquent loan and 0 otherwise. A delinquent loan is one whose status is any of the following:  charged off, late, in grace period, does not meet the credit policy - charged off, and default. Drop "loan_status" variable once you are done creating "delinquency" since it's no longer needed.

#### Feature generation: Once we have our target variable ready, we can move on to generate some features. The model does not learn if you do not provide it with informative hints! 

#### First, we will drop the id columns since they have no inherent value but for book keeping in this case. To simplify the task, we will also drop 'url', 'desc', 'title', 'zip_code', 'emp_title' columns because they contain text information that require more tedious cleaning and natural language processing. Finally, we will drop 'grade' because the information is contained in 'sub_grade'.

__Task__: drop 'id', 'member_id', 'url', 'desc', 'title', 'zip_code', 'emp_title', 'grade' columns

#### Browse through a few rows, you'll notice we have a mix of categorical, date, and numerical variables. Turn all categorical variables into dummy variables using pd.get_dummies().

__Task__: Turn all categorical variables into dummy variables and join them back to the dataframe. Drop the original variable that we used to create the dummy variables. *Hint: There are 11 categorical variables*

#### Now let's clean the date variables. Date variables genernally don't provide much information unless we extract something useful from them. For example from entry/exit dates of a project, we can infer the project duration. We can also extract the month/year from the dates especially when there's seasonality involved in the data. For instance, most people may fail to pay off their loands after 2008 financial crisis. In this data, we will extract the year from the dates.

__Task__: 
        - 1) Make sure date columns contain datetime types 
        - 2) Create 'issued_yr' by extracting year from 'issued_d'
        - 3) Create 'earliest_cr_line_yr' by extracting year from 'earliest_cr_line' 
        - 4) Create 'payment_period' by subtracting 'next_pymnt_d' - 'last_pymnt_d' (make sure you convert timestamp to integers: 54days => 54)
        - 5) Create 'last_credit_pull_yr' by extracting year from 'last_credit_pull_d'
        - 6) Drop all original date variables

#### As a last step, we will fill in the missing values for each column. For numerical columns such as annual income, it's reasonable to assume the missing value will be around the median or mean income. For others such as payment, accounts opened, a missing value may simply indicate no activities happened and the value can be 0. You can also use interpolation (see more) to fill in the missing values. 

__Task__: Fill in missing values at your discretion. include justification of your methods in the comments. 

### Now let's feed the cleaned dataset through the pipeline! 

#### X represents all the predictors - features; y represents the target variable - whether the borrower has been delinquent on the loan

__task__: divide development dataframe into two X and y. X will be a dataframe that contains all predictors. y will be a single column therefore a Series that contains the target variable 'delinquency'

#### For this pipeline, we will experiment with K-Nearest Neighbor algorithm. Similar to what you've done in the case study by finding comparable examples to the case at hand and decide whether you want to lend money or not, this algorithm calculates the distances of each sample to the rest in the feature space, find the K-nearest neighbors to this particular sample, and assign the sample the most common class from its neighbors.

#### There are several variation of the algorithm with some options more efficient than the others -- calculating distance between one and the rest of the samples can get computationally expensive really fast! Read more about KNN [here](https://kevinzakka.github.io/2016/07/13/k-nearest-neighbor/)



__task__: Call pipeline function and store the returned scores in a variable. You do NOT need to modify the pipeline function

In [ ]:
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline
from sklearn.neighbors import KNeighborsClassifier as KNN
from sklearn.model_selection import train_test_split, KFold
from sklearn.model_selection import cross_val_score
import time

def pipeline(X, y):
    
    begin = time.time()
    
    # we will split the development set into training(80%) and testing (20%)
    # 80-20 is a convention. 70-30 is also used. The choice really depends on the base rate of your outcome
    X_train, X_test, y_train, y_test = train_test_split(
            X, y, test_size=0.2, random_state=123, stratify=y)

    # here we use the make_pipeline module to make a pipeline by using standardscaler to standarize
    # the data and feed it through a K-nearest neighbor algorithm
    pipe_kn = make_pipeline(StandardScaler(), 
                            KNN(n_neighbors=30))

    # we will set up a 5-fold cross validation scheme to evaluate the model
    scores = cross_val_score(estimator=pipe_kn,
                             X=X_train,
                             y=y_train,
                             cv=5,
                             scoring='accuracy',
                             n_jobs=1)
    end = time.time()

    print("The model took %d seconds to train." % (end-begin))
    
    return scores

In [ ]:
scores = pipeline(X, y)

#### The model has now been trained, let's see how it performs. There are many metrics you can use to evaluate an algorithm including accuracy, precision, recall, ROC_AUC, and so on. You can read more about it [here](https://machinelearningmastery.com/classification-accuracy-is-not-enough-more-performance-measures-you-can-use/). For simplicity, we will use accuracy -- of all the predictiions we made, how many times did we make a right decision. 

#### To establish comparison, we need a baseline of accuracy. Accuracy is often not a good enough metric by itself. Imagine if you have a very skewed outcome that you are trying to predict -- the outcome only happens 1% of the time. Your model will be very accurate if it keeps giving out negative predictions and would by default reach an accuracy of 1%. So we want our model to have a higher accuracy than the default rate of positive outcomes. 

__Task__: Identify the baseline accuracy. That is ( 1 - (# of delinquency) / (total # of predictions made))

In [ ]:
baseline = 

#### Now let's print out our accuracy obtained from cross-validation by running the following code

In [ ]:
print('CV accuracy scores: %s' % scores)
print('CV accuracy: %.3f +/- %.3f' % (np.mean(scores), np.std(scores)))

### As you can see the average accuracy is way above the baseline rate. Congrats!

#### Bonus step: Now we are happy with the development algorithm, you can train the model on the entire development dataset and use it to predict on the application dataset where the actual outcomes are not yet oberved. Since it can take quite some time to train a model, you can "pickle" the model to avoid having to train the model each time you use it. See more about pickle library [here](https://docs.python.org/3/library/pickle.html).